<a href="https://colab.research.google.com/github/yhp2205/SQL/blob/main/ch_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 7강 - 하나의 테이블에 대한 조작

In [2]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 그룹의 특징 잡기
새로운 테이블을 사용하여 집약함수를 사용하는 방법을 알아보겠습니다.

In [3]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.review;
CREATE TABLE sqldata.review (
    user_id    string
  , product_id string
  , score      numeric
);

INSERT INTO sqldata.review
VALUES
    ('U001', 'A001', 4.0)
  , ('U001', 'A002', 5.0)
  , ('U001', 'A003', 5.0)
  , ('U002', 'A001', 3.0)
  , ('U002', 'A002', 3.0)
  , ('U002', 'A003', 4.0)
  , ('U003', 'A001', 5.0)
  , ('U003', 'A002', 4.0)
  , ('U003', 'A003', 4.0)
;


""


다음과 같이 review 테이블을 생성했습니다.  
COUNT함수는 지정 컬럼의 record 수를 출력하는 함수입니다.  
컬럼 앞에 DISTINCT라고 지정하면 중복을 제외한 수를 출력합니다. 

In [4]:
%%bigquery --project mygcpproject-340112
  -- 7.1 집약 함수를 사용해서 테이블 전체의 특징량을 계산하는 쿼리
SELECT
    COUNT(*) AS total_count
  , COUNT(DISTINCT user_id) AS user_count
  , COUNT(DISTINCT product_id) AS product_count
  , SUM(score) AS sum
  , AVG(score) AS avg
  , MAX(score) AS max
  , MIN(score) AS min
FROM
  sqldata.review
;

,total_count,user_count,product_count,sum,avg,max,min
0,9,3,3,37,4.111111111,5,3


데이터를 더 자세히 분할하고 싶을 때 GROUP BY 구문을 사용하여 데이터를 분류할 키를 지정해주면 그 키에 따른 값을 각각 출력할 수 있습니다. 

In [5]:
%%bigquery --project mygcpproject-340112
  -- 7.2 사용자 기반으로 데이터를 분할하고 집약 함수를 적용하는 쿼리
SELECT
    user_id
  , COUNT(*) AS total_count
  , COUNT(DISTINCT product_id) AS product_count
  , SUM(score) AS sum
  , AVG(score) AS avg
  , MAX(score) AS max
  , MIN(score) AS min
FROM
  sqldata.review
GROUP BY
  user_id
;

,user_id,total_count,product_count,sum,avg,max,min
0,U001,3,3,14,4.666666667,5,4
1,U002,3,3,10,3.333333333,4,3
2,U003,3,3,13,4.333333333,5,4


결과를 보면 user_id에 따라 total_count, product_count, sum, avg, max, mix 값이 각각 출력된 것을 확인할 수 있습니다.

GROUP BY 구문을 사용한 쿼리에서는 GROUP BY 에서 지정한 컬럼만 SELECT 구문의 컬럼으로 지정할 수 있습니다. 앞선 쿼리에서 SELECT 구문에 product_id 혹은 score을 지정할 수 없습니다. 

다음으로 윈도 함수를 사용하여 개별 리뷰 점수와 평균 리뷰 점수의 차이를 구해보도록 하겠습니다.

In [6]:
%%bigquery --project mygcpproject-340112
  -- 7.3 윈도 함수를 사용해 집약 함수의 결과와 원래 값을 동시에 다루는 쿼리
SELECT
    user_id
  , product_id
   -- 개별 리뷰 점수
  , score
   -- 전체 평균 리뷰 점수
  , AVG(score) OVER() AS avg_score
   -- 사용자의 평균 리뷰 점수
  , AVG(score) OVER(PARTITION BY user_id) AS user_avg_score
   -- 개별 리뷰 점수와 사용자 평균 리뷰 점수의 차이
  , score - AVG(score) OVER(PARTITION BY user_id) AS user_avg_score_diff
FROM
  sqldata.review
;

,user_id,product_id,score,avg_score,user_avg_score,user_avg_score_diff
0,U003,A002,4,4.111111111,4.333333333,-0.333333333
1,U003,A003,4,4.111111111,4.333333333,-0.333333333
2,U003,A001,5,4.111111111,4.333333333,0.666666667
3,U001,A002,5,4.111111111,4.666666667,0.333333333
4,U001,A003,5,4.111111111,4.666666667,0.333333333
5,U001,A001,4,4.111111111,4.666666667,-0.666666667
6,U002,A003,4,4.111111111,3.333333333,0.666666667
7,U002,A001,3,4.111111111,3.333333333,-0.333333333
8,U002,A002,3,4.111111111,3.333333333,-0.333333333


다음과 같이 집약함수로 윈도 함수를 사용할 때엔 집약함수 뒤 OVER을 사용하여 윈도함수를 지정합니다. 매개변수에 PARTITION BY를 지정하면 해당 컬럼을 기반으로 그룹화하고 그 앞에 사용된 집약함수를 실행합니다. 

sql의 테이블은 순서라는 개념이 존재하지 않지만 윈도함수를 이용하여 순서를 다루는 것이 쉬워졌습니다. 새로운 data table을 사용하여 순서를 고려한 계산을 해보겠습니다.

In [7]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.popular_products;
CREATE TABLE sqldata.popular_products (
    product_id string
  , category   string
  , score      numeric
);

INSERT INTO sqldata.popular_products
VALUES
    ('A001', 'action', 94)
  , ('A002', 'action', 81)
  , ('A003', 'action', 78)
  , ('A004', 'action', 64)
  , ('D001', 'drama' , 90)
  , ('D002', 'drama' , 82)
  , ('D003', 'drama' , 78)
  , ('D004', 'drama' , 58)
;


""


윈도함수인 OVER 내부에 ORDER BY를 사용하여 윈도 내부에 있는 데이터의 순서를 지정할 수 있습니다.  

In [8]:
%%bigquery --project mygcpproject-340112
  -- 7.4 윈도 함수의 ORDER BY 구문을 사용해 테이블 내부의 순서를 다루는 쿼리
SELECT
    product_id
  , score

    -- 점수 순서로 유일한 순위를 붙임
  , ROW_NUMBER()      OVER(ORDER BY score DESC) AS row
    -- 같은 순위를 허용해서 순위를 붙임
  , RANK()            OVER(ORDER BY score DESC) AS rank
    -- 같은 순위가 있을 때 같은 순위 다음에 있는 순위를 건너뛰고 순위를 붙임
  , DENSE_RANK()      OVER(ORDER BY score DESC) AS dense_rank

    -- 현재 행보다 앞에 있는 행의 값 추출하기
  , LAG(product_id)     OVER(ORDER BY score DESC) AS lag1
  , LAG(product_id, 2)  OVER(ORDER BY score DESC) AS lag2

    -- 현재 행보다 뒤에 있는 행의 값 추출하기
  , LEAD(product_id)    OVER(ORDER BY score DESC) AS lead1
  , LEAD(product_id, 2) OVER(ORDER BY score DESC) AS lead2
FROM sqldata.popular_products
ORDER BY row
;

,product_id,score,row,rank,dense_rank,lag1,lag2,lead1,lead2
0,A001,94,1,1,1,None,None,D001,D002
1,D001,90,2,2,2,A001,None,D002,A002
2,D002,82,3,3,3,D001,A001,A002,D003
3,A002,81,4,4,4,D002,D001,D003,A003
4,D003,78,5,5,5,A002,D002,A003,A004
5,A003,78,6,5,5,D003,A002,A004,D004
6,A004,64,7,7,6,A003,D003,D004,None
7,D004,58,8,8,7,A004,A003,None,None


ORDER BY <컬럼 이름> DESC 를 사용하면 테이블 내부의 상품을 컬럼 값이 높은 순서로 정렬할 수 있습니다.  
ROW_NUMBER 함수는 앞서 정렬한 순서에 순위를 붙이는 함수입니다.  
RANK 함수와 DENSE_RANK 함수를 이용하면 같은 순위의 레코드가 있을 때 순위를 같게 붙인다는 점이 동일하지만, RANK 함수는 다음 숫자를 건너뛰고, DENSE_RANK 함수는 건너뛰지 않는다는 점이 다릅니다.  

LEG 함수와 LEAD 함수는 현재 행을 기준으로 앞이나 뒤의 행의 값을 추출합니다.  
두번째 매개변수를 지정하여 앞뒤 n번째 값을 추출할 수 있습니다. 

In [9]:
%%bigquery --project mygcpproject-340112
  -- ORDER BY 구문과 집약 함수를 조합해서 계산하는 쿼리
SELECT
    product_id
  , score

    -- 점수 순서로 유일한 순위를 붙임
  , ROW_NUMBER() OVER(ORDER BY score DESC) AS row

    -- 순위 상위부터의 누계 점수 계산하기
  , SUM(score)
      OVER(ORDER BY score DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
    AS cum_score

    -- 현재 행과 앞 뒤의 행이 가진 값을 기반으로 평균 점수 계산하기
  , AVG(score)
      OVER(ORDER BY score DESC
        ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING)
    AS local_avg
  
    -- 순위가 높은 상품 ID 추출하기
  , FIRST_VALUE(product_id)
      OVER(ORDER BY score
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
    AS first_value

    -- 순위가 낮은 상품 ID 추출하기
  , LAST_VALUE(product_id)
      OVER(ORDER BY score DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
    AS last_value
FROM sqldata.popular_products
ORDER BY row
;

,product_id,score,row,cum_score,local_avg,first_value,last_value
0,A001,94,1,94,92,D004,D004
1,D001,90,2,184,88.666666667,D004,D004
2,D002,82,3,266,84.333333333,D004,D004
3,A002,81,4,347,80.333333333,D004,D004
4,D003,78,5,425,79,D004,D004
5,A003,78,6,503,73.333333333,D004,D004
6,A004,64,7,567,66.666666667,D004,D004
7,D004,58,8,625,61,D004,D004


### 프레임 지정 구문 (ROWS BETWEEN 'start' AND 'end')
'start', 'end' : 'CURRENT ROW'(현재의 행), 'n PRECEDING'(n행 앞) , 'n FOLLOWING'(n행 뒤) , 'UNBOUNDED FOLLOWING'(이후 행 전부)  
등의 키워드를 지정할 수 있습니다.

PARTITION BY 구문과 ORDER BY 구문을 사용하여 카테고리의 순위를 계산할 수 있습니다. 

In [10]:
%%bigquery --project mygcpproject-340112
  -- 7.7 윈도 함수를 사용해 카테고리들의 순위를 계산하는 쿼리
SELECT
    category
  , product_id
  , score

    -- 카테고리 별로 점수 순서를 정렬하고 유일한 순위를 붙임
  , ROW_NUMBER()
      OVER(PARTITION BY category ORDER BY score DESC)
    AS row


    -- 카테고리별로 같은 순위를 허가하고 순위를 붙임
  , RANK()
      OVER(PARTITION BY category ORDER BY score DESC)
    AS rank

    -- 카테고리별로 같은 순위가 있을 때
    -- 같은 순위 다음에 있는 순위를 건너뛰고 순위를 붙임
  , DENSE_RANK()
      OVER(PARTITION BY category ORDER BY score DESC)
    AS dense_rank
FROM sqldata.popular_products
ORDER BY category, row
;

,category,product_id,score,row,rank,dense_rank
0,action,A001,94,1,1,1
1,action,A002,81,2,2,2
2,action,A003,78,3,3,3
3,action,A004,64,4,4,4
4,drama,D001,90,1,1,1
5,drama,D002,82,2,2,2
6,drama,D003,78,3,3,3
7,drama,D004,58,4,4,4


카테고리별로 순위를 매겼기 때문에 상위 n개를 추출할 수 있습니다.  
윈도 함수에서 WHERE 구문을 적용할 수 없어서 SELECT 구문에서 윈도함수를 이용한 결과를 서브쿼리로 만들고 외부에서 WHERE 구문을 적용해야 합니다. 

In [11]:
%%bigquery --project mygcpproject-340112
  -- 7.8 카테고리들의 순위 상위 2개까지의 상품을 추출하는 쿼리
SELECT *
FROM
  -- 서브 쿼리 내부에서 순위 계산하기
  ( SELECT
        category
      , product_id
      , score
        -- 카테고리별로 점수 순서로 유일한 순위를 붙임
      , ROW_NUMBER()
          OVER(PARTITION BY category ORDER BY score DESC)
        AS rank
    FROM sqldata.popular_products
  ) AS popular_products_with_rank
-- 외부 쿼리에서 순위 활용해 압축하기
WHERE rank <= 2
ORDER BY category, rank
;

,category,product_id,score,rank
0,action,A001,94,1
1,action,A002,81,2
2,drama,D001,90,1
3,drama,D002,82,2


카테고리들의 순위 상위 2개의 상품을 추출하는 쿼리를 작성해보았습니다.  
만약 상위 2개가 아닌 1개를 추출할 경우에 사용할 수 있는 다른 방법도 알아보겠습니다.

In [12]:
%%bigquery --project mygcpproject-340112
  -- 7.9 카테고리별 순위 최상위 상품을 추출하는 쿼리

-- DISTINCT 구문을 사용하여 중복 제거하기
SELECT DISTINCT
    category
    -- 카테고리별로 순위 최상위 상품 ID 추출하기
  , FIRST_VALUE(product_id)
      OVER(PARTITION BY category ORDER BY score DESC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)
    AS product_id
FROM sqldata.popular_products
;

,category,product_id
0,drama,D001
1,action,A001


각 카테고리별 가장 최상위 상품만 추출된 것을 확인할 수 있습니다. 

SQL은 행 기반으로 처리하는 것이 기본이지만,  
결과를 확인할때에는 열 단위로 전개해야 가독성이 높기 때문에  
세로 기반(행)으로 저장된 것을 가로 기반(열)으로 변환하는 것을 알아보겠습니다.

In [13]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS sqldata.daily_kpi;
CREATE TABLE sqldata.daily_kpi (
    dt        string
  , indicator string
  , val       integer
);

INSERT INTO sqldata.daily_kpi
VALUES
    ('2017-01-01', 'impressions', 1800)
  , ('2017-01-01', 'sessions'   ,  500)
  , ('2017-01-01', 'users'      ,  200)
  , ('2017-01-02', 'impressions', 2000)
  , ('2017-01-02', 'sessions'   ,  700)
  , ('2017-01-02', 'users'      ,  250)
;

DROP TABLE IF EXISTS sqldata.purchase_detail_log;
CREATE TABLE sqldata.purchase_detail_log (
    purchase_id integer
  , product_id  string
  , price       integer
);

INSERT INTO sqldata.purchase_detail_log
VALUES
    (100001, 'A001', 3000)
  , (100001, 'A002', 4000)
  , (100001, 'A003', 2000)
  , (100002, 'D001', 5000)
  , (100002, 'D002', 3000)
  , (100003, 'A001', 3000)
;

""


daily_kpi 테이블을 사용하여 날짜별로 지표들의 추이를 볼 수 있도록 열로 변환하겠습니다. 

In [14]:
%%bigquery --project mygcpproject-340112
  -- 7.10 행으로 저장된 지표 값을 열로 변환하는 쿼리
SELECT
    dt
  , MAX(CASE WHEN indicator = 'impressions' THEN val END) AS impressions
  , MAX(CASE WHEN indicator = 'sessions'    THEN val END) AS sessions
  , MAX(CASE WHEN indicator = 'users'       THEN val END) AS users
FROM sqldata.daily_kpi
GROUP BY dt
ORDER BY dt
;

,dt,impressions,sessions,users
0,2017-01-01,1800,500,200
1,2017-01-02,2000,700,250


다음과 같은 쿼리를 이용하여 열로 변환해 보았습니다.  
날짜를 1개의 레코드로 집약할 수 있도록 GROUP BY dt를 사용하였습니다.  
  
추가로 MAX(CASE~) 구문을 사용했는데, CASE를 이용하여 impressions, sessions, users에 해당하는 레코드를 추출하고  
MAX 함수로 가장 높은 값을 추출하도록 했는데 각 날짜별로 지표들이 하나씩만 존재하기 때문에 모든 행을 출력했다고 볼 수 있습니다.

다만 data의 열이 몇개인지 미리 알수 없을때, 앞의 방식을 사용할 수 없기 때문에 데이터를 쉼표 등으로 구분한 문자열로 변환하는 방법을 생각해보겠습니다. purchase_detail_log데이터를 활용합니다.

In [15]:
%%bigquery --project mygcpproject-340112
  -- 7.11 행을 집약해서 쉼표로 구분된 문자열로 변환하기
SELECT
  purchase_id

  -- 상품 ID를 배열에 집약하고 쉼표로 구분된 문자열로 변환하기
  , string_agg(product_id, ',') AS product_ids
  , SUM(price) AS amount
FROM sqldata.purchase_detail_log
GROUP BY purchase_id
ORDER BY purchase_id
;

,purchase_id,product_ids,amount
0,100001,"A001,A002,A003",9000
1,100002,"D001,D002",8000
2,100003,A001,3000


그럼 다시 열로 표현되어있는 값을 행으로 바꾸어보겠습니다.

In [16]:
%%bigquery --project mygcpproject-340112
DROP TABLE IF EXISTS  sqldata.quarterly_sales;
CREATE TABLE sqldata.quarterly_sales (
    year integer
  , q1   integer
  , q2   integer
  , q3   integer
  , q4   integer
);

INSERT INTO sqldata.quarterly_sales
VALUES
    (2015, 82000, 83000, 78000, 83000)
  , (2016, 85000, 85000, 80000, 81000)
  , (2017, 92000, 81000, NULL , NULL )
;


""


다음과 같은 가로 기반 데이터의 특징은 데이터의 수가 고정되어 있다는 것입니다. 데이터 수와 같은 수의 일련번호를 갖는 피벗 테이블을 만들고 CROSS JOIN 구문을 사용하면 됩니다.

In [ ]:
%%bigquery --project mygcpproject-340112
  -- 7.12 일련 번호를 가진 피벗 테이블을 사용해 행으로 변환하는 쿼리
SELECT
    q.year
  -- Q1에서 Q4까지의 레이블 이름 출력하기
  , CASE
      WHEN p.idx = 1 THEN 'q1'
      WHEN p.idx = 2 THEN 'q2'
      WHEN p.idx = 3 THEN 'q3'
      WHEN p.idx = 4 THEN 'q4'
    END AS quarter
  -- Q1에서 Q4까지의 매출 출력하기
  , CASE
  